In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle

### SOC Database (https://zenodo.org/record/6611475)

The dataset compiles 840 georeferenced SOC measurements over a 26-ha agricultural field located in southern Ontario, Canada with a sampling density of ~32 points per ha. As SOC is influenced by site topography (i.e., slope and landscape position), each point of the database was associated with a wide range of topographic derivatives. The columns include sample ID, SOC measurement, latitude, Longitude, NDVI values, as well as a set of 54 topographic derivatives (i.e., primary and secondary - see metadat.pdf attached file) with a spatial resolution of a 5 m.  

In [2]:
# We have used open SOC base for training the ML model and predict the SOC based on NDVI for particular field

socDB = pd.read_csv('SOC_Database.csv', sep=',')
display(socDB.head())
# checking the data
display(socDB.columns)
display(socDB.shape)
display(socDB.isnull().sum().any())
display(socDB.duplicated().any())
display(socDB.dtypes == 'str')

,SampleID,SOC (%),Catch,Conv,Elev,DevME A,DevME B,DevME C,DevME D,DiffME A,...,TPI,TRI,TWI,VDepth,VIS,NDVI_max,NDVI_median,NDVI_sd,X (DD),Y (DD)
0,1,4.88,105.560303,-1.695651,397.309113,-0.242336,-0.019693,-0.416003,-1.223508,-0.042645,...,-0.025165,0.059154,8.648174,9.705094,98.230995,0.876076,0.694713,0.164614,-80.269163,43.707232
1,2,4.76,11756.406250,-12.779182,397.207977,-0.778743,-0.452079,-0.494462,-1.258175,-0.106777,...,-0.180933,0.021448,14.400584,10.325721,98.319786,0.915627,0.613926,0.210947,-80.269013,43.707365
2,3,5.10,2824.329590,-0.521328,397.390015,-0.171732,-0.254896,-0.437166,-1.231417,-0.031157,...,-0.111542,0.037374,12.385666,9.450062,98.371185,0.927210,0.646960,0.234737,-80.268867,43.707490
3,4,5.70,5295.354492,3.123698,397.513275,0.009115,0.044004,-0.394856,-1.225852,0.001653,...,0.000611,0.031468,13.188428,8.735661,98.379845,0.915750,0.711604,0.183207,-80.268703,43.707638
4,5,5.11,6871.969727,2.661578,397.637207,0.072724,0.113439,-0.366388,-1.215629,0.014545,...,0.057567,0.041576,13.167877,8.526403,98.394020,0.934996,0.651558,0.271645,-80.268508,43.707771


Index(['SampleID', 'SOC (%)', 'Catch', 'Conv', 'Elev', 'DevME A', 'DevME B',
       'DevME C', 'DevME D', 'DiffME A', 'DiffME B', 'DiffME C', 'DiffME D',
       'EDF MID', 'EDF NE', 'EDF NW', 'EDF SE', 'EDF SW', 'EDF E', 'EDF N',
       'Gcurv', 'Hillshade', 'LS', 'MDM A', 'MDM B', 'MDM C', 'MDMS A',
       'MDMS B', 'MDMS C', 'MED A', 'MED B', 'MED C', 'MEDS A', 'MEDS B',
       'MEDS C', 'MRRTF', 'MRVBF', 'MSP', 'MSTPI', 'NormH', 'Plan', 'Pro',
       'RSP', 'SLength', 'SlopeH', 'Slope', 'SPI', 'StanH', 'SVF', 'SWI',
       'TCurv', 'TPI', 'TRI', 'TWI', 'VDepth', 'VIS', 'NDVI_max',
       'NDVI_median', 'NDVI_sd', 'X (DD)', 'Y (DD)'],
      dtype='object')

(840, 61)

False

False

SampleID       False
SOC (%)        False
Catch          False
Conv           False
Elev           False
               ...  
NDVI_max       False
NDVI_median    False
NDVI_sd        False
X (DD)         False
Y (DD)         False
Length: 61, dtype: bool

In [3]:
# prepare data for ML modeling
x = socDB.drop(['SampleID','SOC (%)'], 1)
y = socDB['SOC (%)']
display(x.shape, y.shape)

# splitting data for training and testing
X_train = x[:700]
y_train = y[:700]
X_test = x[700:]
y_test = y[700:]

display(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

<ipython-input-3-f5efe0905ebc>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x = socDB.drop(['SampleID','SOC (%)'], 1)


(840, 59)

(840,)

(700, 59)

(700,)

(140, 59)

(140,)

In [4]:
# Training model using simple linear regression
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_train, y_train)

print(clf.coef_)
print(clf.intercept_)

train_accuracy = clf.score(X_train, y_train)
test_accuracy = clf.score(X_test, y_test)

print(train_accuracy)
print(test_accuracy)

train_predict = clf.predict(X_train)

train_predictions = pd.DataFrame({'true_SOC%': y_train, 'predicted_SOC%': train_predict})
train_predictions.head()


test_predict = clf.predict(X_test)
test_predictions = pd.DataFrame({'true_SOC%': y_test, 'predicted_SOC%': test_predict})

print("-------Training Predictions--------")
print(train_predictions.head())


print("-------Testing Predictions--------")
print(test_predictions.head())

[-1.91928514e-05  1.92881216e-03  2.10953744e+00 -2.15581429e-03
 -8.13729161e-01 -3.87019173e-01 -1.23077024e+01  9.51798209e+00
 -4.59731112e-02  8.24202478e-01 -3.93268382e-02 -8.00061102e-03
 -1.51064248e-01  8.86280927e-02 -1.41731535e-02  1.70017541e-02
  1.70592630e-01  8.73285258e-02 -1.27318797e+02  6.79042555e-01
 -3.03851035e+00  4.22216096e-01  1.38149001e-01 -1.74342238e-01
  6.49690852e-03 -1.67495417e-03  2.77177074e-03  2.72772009e-02
 -3.31504930e-01  5.50978352e-01 -6.14792253e-04 -2.29028769e-03
 -1.60885330e-03 -6.32140712e-02 -8.53141246e-02  1.27089325e-01
 -1.12976082e+00  2.89793569e+00  5.12118051e-01 -1.19683170e+02
  1.58396630e+00 -1.49988622e-05 -2.37187678e-01  4.58230953e+01
  1.42090692e-03 -1.42885234e-01 -5.24938686e+02 -1.05900614e-01
  2.22301529e+04 -5.47956940e-01 -9.55873627e+00  3.32599598e-02
  2.51624452e-01  3.57258851e-01 -7.16934805e+00 -1.27138025e+00
 -1.91831646e+00  5.27960323e+02  1.81549675e+03]
-37453.443620437254
0.6818351594493779
-

In [18]:
import requests
import warnings
warnings.filterwarnings('ignore')
response = requests.get("http://api.agromonitoring.com/stats/1.0/12363b4c180/635cf25a176fe6831443f2df?appid=b58f66ea030f42fad0c0d3c651b4fbe5")
ndvi = response.json()

# Satelite data
NDVI_sd = ndvi['std']
NDVI_max = ndvi['max']
NDVI_median = ndvi['median']

# Field data (collected from soc database)
field = x[2:4]
"Combining field data with NDVI data"
field['NDVI_sd'] = NDVI_sd
field['NDVI_max'] = NDVI_max
field['NDVI_median'] = NDVI_median


# prediction of soil% organic carbon for selected fields
socPercent = clf.predict(field)
print("Calculated SOC%", socPercent)

Calculated SOC% [10.77488994 10.41984578]


In [27]:
"""
The SOC stock was calculated based on the SOC% and Bulk_Density of soil profile
SOC_stock = Depth * BulckDensity * SOC%
"""
LandSize = 10000 # in m^2 one hectare
Depth = 0.1 # in meters
BulckDensity = 1.4 # in g/cm^3
SOCPercent = 10.8 # Calculated through ML

SOCStock = round((LandSize * Depth * BulckDensity * SOCPercent)/ 100) # Per hectare in tons
TotalCC = 1 * SOCStock # 1 Credit for 1 ton of Carbon stock

print("Total Carbon stock per hectare in tons :", SOCStock)
print("Total Carbon credits eligible : ", round(TotalCC))

Total Carbon stock per hectare in tons : 151
Total Carbon credits eligible :  151
